In [1]:
%pylab inline
import numexpr as ne
import seaborn as sns
from ipywidgets import interact
import time
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('../../..')

In [4]:
%gui qt

In [5]:
from spiketag.base.Binload import bload
from spiketag.view import wave_view

In [6]:
nCh = 64
fs  = 30e3
bf  = bload(nCh, fs)

In [7]:
bf.load('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_160_30K_bank0and1_25:5:2017.bin')

2017-07-06 00:48:51,206 - spiketag - INFO - #############  load data  ###################
2017-07-06 00:48:51,207 - spiketag - INFO - /Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_160_30K_bank0and1_25:5:2017.bin loaded, it contains: 
2017-07-06 00:48:51,207 - spiketag - INFO - 18000000 * 64 points (4608000000 bytes) 
2017-07-06 00:48:51,208 - spiketag - INFO - 64 channels with sampling rate of 30000.0000 
2017-07-06 00:48:51,209 - spiketag - INFO - 600.000 secs (10.000 mins) of data
2017-07-06 00:48:51,210 - spiketag - INFO - #############################################


In [ ]:
data = bf.npmm.reshape(-1, nCh)

In [ ]:
spks = np.fromfile('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_160_30K_bank0and1_25:5:2017.bin.spk', dtype=int32)

In [ ]:
wv = wave_view(fs=30e3, data=data, spks=spks, chs=[36, 37, 38, 39])

In [ ]:
wv.show()

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f wv.__init__ wv.__init__(data=data, spks=spks, chs=[i for i in range(32) if i % 5 == 0])